In [1]:
""" Add downstream PFAFIDs to the merged hydrobasin csv file.
-------------------------------------------------------------------------------
Create a csv file with all PFAFID's downstream.

Revisited to apply normalization to database structure. 

Author: Rutger Hofste
Date: 20170823
Kernel: python27 -> python35
Docker: rutgerhofste/gisdocker:ubuntu16.04

Args:
    TESTING (Boolean) : Toggle testing case.
    SCRIPT_NAME (string) : Script name.
    OUTPUT_VERSION (integer) : output version.
    DATABASE_ENDPOINT (string) : RDS or postGreSQL endpoint.
    DATABASE_NAME (string) : Database name.
    TABLE_NAME_AREA_30SPFAF06 (string) : Table name used for areas. Must exist
        on same database as used in rest of script.
    S3_INPUT_PATH_RIVERDISCHARGE (string) : AWS S3 input path for 
        riverdischarge.    
    S3_INPUT_PATH_DEMAND (string) : AWS S3 input path for 
        demand.    

"""
SCRIPT_NAME = "Y2017M08D23_RH_Downstream_V01"
OUTPUT_VERSION = 1

S3_INPUT_PATH = "s3://wri-projects/Aqueduct30/processData/Y2017M08D22_RH_Upstream_V01/output_V04/"

INPUT_FILENAME = "hybas_lev06_v1c_merged_fiona_upstream_V01.csv"
OUTPUT_FILENAME = "hybas_lev06_v1c_merged_fiona_upstream_downstream_V01.csv"


ec2_input_path = "/volumes/data/{}/input_V{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION)
ec2_output_path = "/volumes/data/{}/output_V{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION)
s3_output_path = "s3://wri-projects/Aqueduct30/processData/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)

print("Input ec2: " + ec2_input_path,
      "\nInput s3: " + S3_INPUT_PATH ,
      "\nOutput ec2: " + ec2_output_path,
      "\nOutput s3: " + s3_output_path)

Input ec2: /volumes/data/Y2017M08D23_RH_Downstream_V01/input_V01 
Input s3: s3://wri-projects/Aqueduct30/processData/Y2017M08D22_RH_Upstream_V01/output_V04/ 
Output ec2: /volumes/data/Y2017M08D23_RH_Downstream_V01/output_V01 
Output s3: s3://wri-projects/Aqueduct30/processData/Y2017M08D23_RH_Downstream_V01/output_V01/


In [2]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2018M05D24 UTC 09:26


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [3]:
import pandas as pd
import numpy as np
import os
import ftplib
from ast import literal_eval

In [4]:
!rm -r {ec2_input_path} 
!rm -r {ec2_output_path} 
!mkdir -p {ec2_input_path} 
!mkdir -p {ec2_output_path} 

In [5]:
!aws s3 cp {S3_INPUT_PATH} {ec2_input_path} --recursive

download: s3://wri-projects/Aqueduct30/processData/Y2017M08D22_RH_Upstream_V01/output_V04/hybas_lev06_v1c_merged_fiona_V01.csv to ../../../../data/Y2017M08D23_RH_Downstream_V01/input_V01/hybas_lev06_v1c_merged_fiona_V01.csv
download: s3://wri-projects/Aqueduct30/processData/Y2017M08D22_RH_Upstream_V01/output_V04/hybas_lev06_v1c_merged_fiona_upstream_V01.csv to ../../../../data/Y2017M08D23_RH_Downstream_V01/input_V01/hybas_lev06_v1c_merged_fiona_upstream_V01.csv


In [6]:
def stringToList(string):
    # input format : "[42, 42, 42]" , note the spaces after the commas, in this case I have a list of integers
    string = string[1:len(string)-1]
    try:
        if len(string) != 0: 
            tempList = string.split(", ")
            newList = list(map(lambda x: int(x), tempList))
        else:
            newList = []
    except:
        newList = [-9999]
    return(newList)

In [7]:
df = pd.read_csv(os.path.join(ec2_input_path,INPUT_FILENAME))

In [8]:
df["Upstream_HYBAS_IDs"] = df["Upstream_HYBAS_IDs"].apply(lambda x: stringToList(x))
df["Upstream_PFAF_IDs"] = df["Upstream_PFAF_IDs"].apply(lambda x: stringToList(x))

In [9]:
header = df.dtypes

df["HYBAS_ID2"] = df["HYBAS_ID"]
df = df.set_index(["HYBAS_ID2"])


# testing purposes
#df = df[ 0:200]
df_out = df.copy()


df_out['Downstream_HYBAS_IDs'] = "Nodata"
df_out['Downstream_PFAF_IDs'] = "Nodata"

print(df_out.dtypes)

i = 1
for id in df.index:
    print("item: ", i, " id: ",id)
    i += 1
    writeID = id
    allDownID = []
    allDownPFAF = []    

    sinkHybasID = np.int64(df.loc[id]["NEXT_SINK"])
    sinkPfafID = np.int64(df.loc[sinkHybasID]["PFAF_ID"])


    while id != 0:
              
        series = df.loc[id]
        downId = np.int64(series["NEXT_DOWN"]) # Next down ID
        if downId != 0 :
            downSeries = df.loc[downId]
            pfafID = np.int64(downSeries["PFAF_ID"])
            allDownID.append(downId)
            allDownPFAF.append(pfafID)
            id = downId
        else:
            id = 0
            pass

    df_out.set_value(writeID, 'Downstream_HYBAS_IDs', allDownID)
    df_out.set_value(writeID, 'Downstream_PFAF_IDs',allDownPFAF)

    df_out.set_value(writeID, 'NEXT_SINK_PFAF', sinkPfafID )


df_out.to_csv(outputLocation)

print("done")

Unnamed: 0                int64
HYBAS_ID                  int64
NEXT_DOWN                 int64
NEXT_SINK                 int64
MAIN_BAS                  int64
DIST_SINK               float64
DIST_MAIN               float64
SUB_AREA                float64
UP_AREA                 float64
PFAF_ID                   int64
ENDO                      int64
COAST                     int64
ORDER                     int64
SORT                      int64
Upstream_HYBAS_IDs       object
Upstream_PFAF_IDs        object
Downstream_HYBAS_IDs     object
Downstream_PFAF_IDs      object
dtype: object
item:  1  id:  6060000010
item:  2  id:  6060000200
item:  3  id:  6060000210
item:  4  id:  6060000240
item:  5  id:  6060000250
item:  6  id:  6060000450
item:  7  id:  6060000460
item:  8  id:  6060000580
item:  9  id:  6060000590
item:  10  id:  6060000740
item:  11  id:  6060068100
item:  12  id:  6060068110
item:  13  id:  6060070260
item:  14  id:  6060070270
item:  15  id:  6060070390
item:  16  id:

/opt/anaconda3/envs/python35/lib/python3.5/site-packages/ipykernel_launcher.py:43: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/opt/anaconda3/envs/python35/lib/python3.5/site-packages/ipykernel_launcher.py:44: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/opt/anaconda3/envs/python35/lib/python3.5/site-packages/ipykernel_launcher.py:46: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead



item:  99  id:  6060120370
item:  100  id:  6060120320
item:  101  id:  6060105620
item:  102  id:  6060105580
item:  103  id:  6060096020
item:  104  id:  6060096010
item:  105  id:  6060081020
item:  106  id:  6060080920
item:  107  id:  6060079970
item:  108  id:  6060079860
item:  109  id:  6060079280
item:  110  id:  6060079250
item:  111  id:  6060076650
item:  112  id:  6060076610
item:  113  id:  6060102250
item:  114  id:  6060102330
item:  115  id:  6060075090
item:  116  id:  6060075000
item:  117  id:  6060073070
item:  118  id:  6060073150
item:  119  id:  6060071160
item:  120  id:  6060071120
item:  121  id:  6060113870
item:  122  id:  6060113940
item:  123  id:  6060115330
item:  124  id:  6060115340
item:  125  id:  6060118140
item:  126  id:  6060118200
item:  127  id:  6060120040
item:  128  id:  6060120030
item:  129  id:  6060134450
item:  130  id:  6060134460
item:  131  id:  6060134890
item:  132  id:  6060134790
item:  133  id:  6060083060
item:  134  id:  606

item:  400  id:  6060301750
item:  401  id:  6060301850
item:  402  id:  6060404900
item:  403  id:  6060404790
item:  404  id:  6060418360
item:  405  id:  6060418530
item:  406  id:  6060280410
item:  407  id:  6060280400
item:  408  id:  6060439370
item:  409  id:  6060439360
item:  410  id:  6060316140
item:  411  id:  6060316270
item:  412  id:  6060278200
item:  413  id:  6060278070
item:  414  id:  6060400740
item:  415  id:  6060400660
item:  416  id:  6060405310
item:  417  id:  6060405320
item:  418  id:  6060410440
item:  419  id:  6060410430
item:  420  id:  6060408590
item:  421  id:  6060408450
item:  422  id:  6060410950
item:  423  id:  6060410660
item:  424  id:  6060213670
item:  425  id:  6060213770
item:  426  id:  6060342520
item:  427  id:  6060342690
item:  428  id:  6060337190
item:  429  id:  6060337320
item:  430  id:  6060294270
item:  431  id:  6060294360
item:  432  id:  6060255000
item:  433  id:  6060255010
item:  434  id:  6060254550
item:  435  id:  606

item:  743  id:  6060384900
item:  744  id:  6060379950
item:  745  id:  6060379960
item:  746  id:  6060390310
item:  747  id:  6060390420
item:  748  id:  6060399630
item:  749  id:  6060399620
item:  750  id:  6060411440
item:  751  id:  6060411450
item:  752  id:  6060408150
item:  753  id:  6060408040
item:  754  id:  6060443080
item:  755  id:  6060442950
item:  756  id:  6060449500
item:  757  id:  6060449170
item:  758  id:  6060458930
item:  759  id:  6060459050
item:  760  id:  6060451380
item:  761  id:  6060451390
item:  762  id:  6060476790
item:  763  id:  6060476800
item:  764  id:  6060462190
item:  765  id:  6060462200
item:  766  id:  6060480090
item:  767  id:  6060480100
item:  768  id:  6060481840
item:  769  id:  6060481630
item:  770  id:  6060486940
item:  771  id:  6060486950
item:  772  id:  6060490170
item:  773  id:  6060490160
item:  774  id:  6060477950
item:  775  id:  6060478100
item:  776  id:  6060498620
item:  777  id:  6060498490
item:  778  id:  606

...

item:  15384  id:  3060478140
item:  15385  id:  3060522830
item:  15386  id:  3060522940
item:  15387  id:  3060411130
item:  15388  id:  3060411560
item:  15389  id:  3060419770
item:  15390  id:  3060419930
item:  15391  id:  3060593150
item:  15392  id:  3060593200
item:  15393  id:  3060601680
item:  15394  id:  3060601790
item:  15395  id:  3060607050
item:  15396  id:  3060607060
item:  15397  id:  3060547220
item:  15398  id:  3060547400
item:  15399  id:  3060602560
item:  15400  id:  3060602570
item:  15401  id:  3060611990
item:  15402  id:  3060611980
item:  15403  id:  3060588280
item:  15404  id:  3060588440
item:  15405  id:  3060613820
item:  15406  id:  3060613940
item:  15407  id:  3060605740
item:  15408  id:  3060605880
item:  15409  id:  3060604230
item:  15410  id:  3060604190
item:  15411  id:  3060624900
item:  15412  id:  3060624780
item:  15413  id:  3060495980
item:  15414  id:  3060495970
item:  15415  id:  3060633280
item:  15416  id:  3060633290
item:  154

item:  15710  id:  3060168230
item:  15711  id:  3060170710
item:  15712  id:  3060170450
item:  15713  id:  3060201050
item:  15714  id:  3060200730
item:  15715  id:  3060238580
item:  15716  id:  3060238760
item:  15717  id:  3060245370
item:  15718  id:  3060244870
item:  15719  id:  3060255200
item:  15720  id:  3060255340
item:  15721  id:  3060257860
item:  15722  id:  3060257700
item:  15723  id:  3060258480
item:  15724  id:  3060258220
item:  15725  id:  3060264560
item:  15726  id:  3060264710
item:  15727  id:  3060268880
item:  15728  id:  3060269050
item:  15729  id:  3060282160
item:  15730  id:  3060282370
item:  15731  id:  3060297840
item:  15732  id:  3060297700
item:  15733  id:  3060297180
item:  15734  id:  3060297030
item:  15735  id:  3060299300
item:  15736  id:  3060299510
item:  15737  id:  3060299520
item:  15738  id:  3060299320
item:  15739  id:  3060303860
item:  15740  id:  3060303460
item:  15741  id:  3060287750
item:  15742  id:  3060287680
item:  157

item:  15984  id:  3060590250
item:  15985  id:  3060581920
item:  15986  id:  3060581930
item:  15987  id:  3060604340
item:  15988  id:  3060604350
item:  15989  id:  3060569980
item:  15990  id:  3060570200
item:  15991  id:  3060580480
item:  15992  id:  3060580600
item:  15993  id:  3060612400
item:  15994  id:  3060612520
item:  15995  id:  3060618970
item:  15996  id:  3060618860
item:  15997  id:  3060009140
item:  15998  id:  3060009320
item:  15999  id:  3060009410
item:  16000  id:  3060009420
item:  16001  id:  3060009430
item:  16002  id:  3060009440
item:  16003  id:  3060009470
item:  16004  id:  3060009480
item:  16005  id:  3060009510
item:  16006  id:  3060009520
item:  16007  id:  3060009670
item:  16008  id:  3060075240
item:  16009  id:  3060075250
item:  16010  id:  3060078770
item:  16011  id:  3060078780
item:  16012  id:  3060109330
item:  16013  id:  3060109340
item:  16014  id:  3060117800
item:  16015  id:  3060117790
item:  16016  id:  3060125440
item:  160

item:  16272  id:  3060017020
item:  16273  id:  3060017070
item:  16274  id:  3060017080
item:  16275  id:  3060017250
item:  16276  id:  3060017260
item:  16277  id:  3060017390
item:  16278  id:  3060017400
item:  16279  id:  3060017430
item:  16280  id:  3060017440
item:  16281  id:  3060017490
item:  16282  id:  3060017500
item:  16283  id:  3060017570
item:  16284  id:  3060017720
item:  16285  id:  3060017730
item:  16286  id:  3060017760
item:  16287  id:  3060017770
item:  16288  id:  3060017800
item:  16289  id:  3060017810
item:  16290  id:  3060017820
item:  16291  id:  3060017830
item:  16292  id:  3060017940
item:  16293  id:  3060017950
item:  16294  id:  3060018390
item:  16295  id:  3060018400
item:  16296  id:  3060018920
item:  16297  id:  3060018930
item:  16298  id:  3060019350
item:  16299  id:  3060019360
item:  16300  id:  3060019510
item:  16301  id:  3060019520
item:  16302  id:  3060019750
item:  16303  id:  3060019760
item:  16304  id:  3060019900
item:  163

NameError: name 'outputLocation' is not defined

In [ ]:
def concatenateHybas(row):
    return row["Downstream_HYBAS_IDs"] + row["Upstream_HYBAS_IDs"] + [row["HYBAS_ID"]]

In [ ]:
df_out['Basin_HYBAS_IDs'] = df_out.apply(concatenateHybas, axis=1)

In [ ]:
def concatenatePFAF(row):
    return row["Downstream_PFAF_IDs"] + row["Upstream_PFAF_IDs"] + [row["PFAF_ID"]]

In [ ]:
df_out['Basin_PFAF_IDs'] = df_out.apply(concatenatePFAF, axis=1)

In [ ]:
df_out.tail()

In [ ]:
df_out.to_csv(os.path.join(ec2_output_path,OUTPUT_FILENAME))

In [ ]:
!aws s3 cp {ec2_output_path} {s3_output_path} --recursive

In [ ]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

Previous Runs:  
